In [134]:
import pandas as pd 
import matplotlib.pyplot as plt
from statsmodels.stats.weightstats import ztest
from scipy.stats import chi2_contingency
import numpy as np
float_formatter = "{:2.2f}".format
np.set_printoptions(formatter={'float_kind':float_formatter})
from collections import Counter

In [2]:
study_original_df = pd.read_excel("./data/studyGroup-original.xlsx")
study_scores_df = pd.read_excel("./data/studyGroup-scores.xls")
control_scores_df = pd.read_excel("./data/control-scoring.xlsx")

In [3]:
study_scores_df.head()

,ID,Gender,Age,Employment,Marital,children,MinorChildren,AcademicalLevel,DiabetesType,DurationDiabetes,...,Energy27,BlueDownhearted28,Wornout29,happy30,Tired31,P32hysicalEmotionOnSocial2,SickEasier33,AsEqual34,worsen35,ExcellentHealth36
0,1,0,47,3,2,1,5,4,2,12,...,80,100,60,100,40,50,50,25,75,75
1,2,0,64,2,2,1,1,5,2,26,...,100,100,60,100,40,50,100,75,100,100
2,3,0,67,2,4,1,0,5,2,15,...,100,60,60,100,60,100,50,75,75,100
3,4,0,41,3,2,1,4,2,2,7,...,60,80,40,100,40,75,50,25,75,100
4,5,0,57,2,2,1,0,4,2,17,...,60,100,60,100,40,25,50,75,50,75


In [4]:
control_scores_df.head()

,ID,gender,age,marital,Employment,AcademicalLevel,DiabetesType,DurationDiabetes,MedicineType,Adherence,...,Energy27,BlueDownhearted28,Wornout29,happy30,Tired31,P32hysicalEmotionOnSocial2,SickEasier33,AsEqual34,worsen35,ExcellentHealth36
NaN,1,1,63,3,1,5,2,3,1,1,...,60,60,60,40,60,75,75,25,50,75
NaN,2,0,16,1,0,4,1,6,0,1,...,60,100,40,60,40,100,25,50,50,25
NaN,3,1,52,2,0,4,2,10,1,1,...,60,20,0,80,0,75,50,25,25,75
NaN,4,1,52,2,0,4,2,22,0,1,...,80,80,60,40,60,75,50,25,50,75
NaN,5,0,44,2,1,5,2,16,0,0,...,80,40,60,60,60,75,25,50,50,75


In [5]:
#print(study_scores_df.describe())

In [6]:
#print(study_scores_df.dtypes)

In [7]:
def adjust_idx(l):
    return [item-1 for item in l]

In [46]:
study_socio_demog = study_scores_df.iloc[:, :17]
control_socio_demog = control_scores_df.iloc[:, :11]

study_domains = study_scores_df.iloc[:,17:]
control_domains = control_scores_df.iloc[:,11:]


In [47]:
study_domains_list = []
control_domains_list = []

In [48]:
study_phys_func = study_domains.iloc[:,2:12]
study_domains_list.append(study_phys_func)

study_rol_lims_health = study_domains.iloc[:, adjust_idx([13, 14, 15, 16])]
study_domains_list.append(study_rol_lims_health)

study_rol_lims_emotion = study_domains.iloc[:, adjust_idx([17, 18, 19])]
study_domains_list.append(study_rol_lims_emotion)

study_energy = study_domains.iloc[:, adjust_idx([23, 27, 29, 31])]
study_domains_list.append(study_energy)

study_emoti_well_being = study_domains.iloc[:, adjust_idx([24, 25, 26, 28, 30])]
study_domains_list.append(study_emoti_well_being)

study_social_fuctioning =  study_domains.iloc[:, adjust_idx([20, 32])]
study_domains_list.append(study_social_fuctioning)

study_pain = study_domains.iloc[:, adjust_idx([21, 22])]
study_domains_list.append(study_pain)

study_general_health = study_domains.iloc[:, adjust_idx([1, 33, 34, 35, 36])]
study_domains_list.append(study_general_health)


control_phys_func = control_domains.iloc[:,2:12]
control_domains_list.append(control_phys_func)

control_rol_lims_health = control_domains.iloc[:, adjust_idx([13, 14, 15, 16])]
control_domains_list.append(control_rol_lims_health)

control_rol_lims_emotion = control_domains.iloc[:, adjust_idx([17, 18, 19])]
control_domains_list.append(control_rol_lims_emotion)

control_energy = control_domains.iloc[:, adjust_idx([23, 27, 29, 31])]
control_domains_list.append(control_energy)

control_emoti_well_being = control_domains.iloc[:, adjust_idx([24, 25, 26, 28, 30])]
control_domains_list.append(control_emoti_well_being)

control_social_fuctioning =  control_domains.iloc[:, adjust_idx([20, 32])]
control_domains_list.append(control_social_fuctioning)

control_pain = control_domains.iloc[:, adjust_idx([21, 22])]
control_domains_list.append(control_pain)

control_general_health =  control_domains.iloc[:, adjust_idx([1, 33, 34, 35, 36])]
control_domains_list.append(control_general_health)


In [49]:
print("    mean - std           mean - std       ****      Z-Score   -   Pvalue")
print("")
for study_domain, control_domain in zip (study_domains_list, control_domains_list):
    avg_study_domain = study_domain.mean(axis = 1)
    avg_control_domain = control_domain.mean(axis =1)
    zval, pval = ztest(avg_study_domain,avg_control_domain )
    print("{:06.3f} \u00B1 {:06.3f}       {:06.3f} \u00B1 {:07.3f}    ****     {: 07.3f}       {: 06.3f}".format(np.mean(avg_study_domain) ,
                                                                                np.std(avg_study_domain),
                                                                                np.mean(avg_control_domain),np.std(avg_control_domain),
                                                                               zval, pval ))

    mean - std           mean - std       ****      Z-Score   -   Pvalue

23.500 ± 26.961       74.333 ± 026.637    ****     -10.302        0.000
04.583 ± 16.765       65.417 ± 038.213    ****     -11.198        0.000
22.222 ± 37.843       78.889 ± 034.409    ****     -08.510        0.000
51.667 ± 14.073       50.083 ± 017.041    ****      00.550        0.582
74.200 ± 15.840       61.667 ± 017.286    ****      04.106        0.000
53.125 ± 32.164       78.333 ± 026.405    ****     -04.653        0.000
61.250 ± 34.048       75.500 ± 026.743    ****     -02.528        0.011
61.250 ± 14.652       55.417 ± 016.363    ****      02.040        0.041


In [54]:
control_socio_demog

,ID,gender,age,marital,Employment,AcademicalLevel,DiabetesType,DurationDiabetes,MedicineType,Adherence,Smoking
NaN,1,1,63,3,1,5,2,3,1,1,0
NaN,2,0,16,1,0,4,1,6,0,1,0
NaN,3,1,52,2,0,4,2,10,1,1,0
NaN,4,1,52,2,0,4,2,22,0,1,0
NaN,5,0,44,2,1,5,2,16,0,0,0
NaN,6,1,52,2,0,4,2,8,0,1,0
NaN,7,1,55,2,0,3,2,10,1,0,0
NaN,8,1,37,1,1,5,2,6,1,1,0
NaN,9,1,50,4,0,5,2,15,1,1,0
NaN,10,1,59,1,0,3,2,3,1,1,0


In [51]:
study_socio_demo_numerical = []
control_socio_demo_numerical = [] 

study_socio_demo_categorical = [] 
control_socio_demo_categorical = [] 

In [52]:
study_gender = study_socio_demog.iloc[:, 1]
study_socio_demo_categorical.append(study_gender)

study_age = study_socio_demog.iloc[:, 2]
study_socio_demo_numerical.append(study_age)

study_employment = study_socio_demog.iloc[:, 3]
study_socio_demo_categorical.append(study_employment)

study_marital = study_socio_demog.iloc[:,4]
study_socio_demo_categorical.append(study_marital)

study_child = study_socio_demog.iloc[:, 5]
study_socio_demo_categorical.append(study_child)

study_minor_chi = study_socio_demog.iloc[:, 6]
study_socio_demo_numerical.append(study_minor_chi)

study_acad_level = study_socio_demog.iloc[:, 7]
study_socio_demo_numerical.append(study_acad_level)

study_diab_type = study_socio_demog.iloc[:, 8]
study_socio_demo_numerical.append(study_diab_type)

study_duration_diab = study_socio_demog.iloc[:, 9]
study_socio_demo_numerical.append(study_duration_diab)

study_medType = study_socio_demog.iloc[:, 10]
study_socio_demo_categorical.append(study_medType)

study_adhere = study_socio_demog.iloc[:, 11]
study_socio_demo_categorical.append(study_adhere)

In [56]:
control_gender = control_socio_demog.iloc[:, 1]
control_socio_demo_categorical.append(control_gender)

control_age = control_socio_demog.iloc[:, 2]
control_socio_demo_numerical.append(control_age)

control_employment = control_socio_demog.iloc[:, 3]
control_socio_demo_categorical.append(control_employment)

control_marital = control_socio_demog.iloc[:,4]
control_socio_demo_categorical.append(control_marital)

# control_child = control_socio_demog.iloc[:, 5]
# control_socio_demo_categorical.append(control_child)

# control_minor_chi = control_socio_demog.iloc[:, 6]
# control_socio_demo_numerical.append(control_minor_chi)

control_acad_level = control_socio_demog.iloc[:, 5]
control_socio_demo_numerical.append(control_acad_level)

control_diab_type = control_socio_demog.iloc[:, 6]
control_socio_demo_numerical.append(control_diab_type)

control_duration_diab = control_socio_demog.iloc[:, 7]
control_socio_demo_numerical.append(control_duration_diab)

control_medType = control_socio_demog.iloc[:, 8]
control_socio_demo_categorical.append(control_medType)

control_adhere = control_socio_demog.iloc[:, 9]
control_socio_demo_categorical.append(control_adhere)

In [129]:
a = study_employment
b = control_employment

In [130]:
c_a = Counter(a)
c_b = Counter(b)

uniq_a = [a for a, b in c_a.items()]
uniq_b = [a for a, b in c_b.items()]
uniq = sorted(list(set(uniq_a + uniq_b)))
print(uniq)
print(c_a)
print(c_b)

[1, 2, 3, 4]
Counter({2: 33, 3: 22, 1: 5})
Counter({2: 43, 1: 11, 3: 4, 4: 2})


In [131]:
freq_a = []
freq_b = []
for element in uniq:
    freq_a.append(c_a[element])
    freq_b.append(c_b[element])

In [139]:
print(freq_a)
print(freq_b)
obs = np.array([freq_a, freq_b])


[5, 33, 22, 0]
[11, 43, 4, 2]
[[ 5 33 22  0]
 [11 43  4  2]]


In [141]:
s, p, _, _ = chi2_contingency(obs)
print (s,p)

18.0273279352 0.000434178080485
